In [2]:
import pandas as pd
import mysql
import mysql.connector as conn
from mysql.connector import Error
import pymongo
import logging

In [3]:
#creating logging file in we store our activity
logging.basicConfig(filename="manage.log", level = logging.INFO)

In [4]:
try:
    logging.info('trying to read dataset and store in variable name df')
    df = pd.read_csv('FitBit data.csv')
    logging.info('we have succefully read the dataset')
    
except Error as e:
    logging.error('problem while reading the dataset',e)

In [5]:
df.head()

,Id,ActivityDate,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories
0,1503960366,3/25/2016,11004,7.11,7.11,0.0,2.57,0.46,4.07,0.0,33,12,205,804,1819
1,1503960366,3/26/2016,17609,11.55,11.55,0.0,6.92,0.73,3.91,0.0,89,17,274,588,2154
2,1503960366,3/27/2016,12736,8.53,8.53,0.0,4.66,0.16,3.71,0.0,56,5,268,605,1944
3,1503960366,3/28/2016,13231,8.93,8.93,0.0,3.19,0.79,4.95,0.0,39,20,224,1080,1932
4,1503960366,3/29/2016,12041,7.85,7.85,0.0,2.16,1.09,4.61,0.0,28,28,243,763,1886


In [6]:
try:
    logging.info('trying to make connection to mysql database and create a database name fit_data')
    mydb = conn.connect(host='localhost',user = 'root',password = 'Rajput@77')
    if mydb.is_connected():
        cursor = mydb.cursor()
        cursor = cursor.execute("create database if not exists fit_data")
        mydb.commit()
    logging.info('we have succesfully made connection to database') 
except Error as e:
    logging.error('problem while connecting to database',e)

In [7]:
try:
    logging.info('creating a table name table1 in database fit_data')
    cursor = mydb.cursor()
    cursor.execute("create table if not exists fit_data.table1(Id int, ActivityDate varchar(17), TotalSteps int, TotalDistance float(17), TrackerDistance float(17),LoggedActivitiesDistance float(17), VeryActiveDistance float(17),ModeratelyActiveDistance float(17), LightActiveDistance float(17),SedentaryActiveDistance float(17), VeryActiveMinutes int, FairlyActiveMinutes int , LightlyActiveMinutes int , SedentaryMinutes int, Calories int )")    
    mydb.commit()
    logging.info('we have create table in our database successfully')
except Error as e:
    logging.error('proble while creating table in databse',e)
    

In [8]:
#before inserting data we have to convert our id column type to bigint because it may giv error
cursor.execute("ALTER TABLE fit_data.table1 MODIFY Id BIGINT")

In [9]:
try:
    logging.info('inserting data in table1')
    for i,row in df.iterrows():
        sql = "insert into fit_data.table1 values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        cursor.execute(sql,tuple(row))
        mydb.commit()
    logging.info('data inserted succesfully')
    
except Error as e:
    logging.error('problem while inserting data in table1',e)

In [10]:
try:
    logging.info('convert datatype of a column and store it in a new colummn name new_ActivityDate')
    df['new_ActivityDate'] = pd.to_datetime(df.ActivityDate)
    df.head(2)
    logging.info('we have converted datatype of column succesfully')
except Error as e:
    logging.error('problem while converting datatype of a column')

In [99]:
#df.drop(['new_ActivityDate'], axis=1,inplace = True)

In [12]:
#df.dtypes

In [15]:
try:
    logging.info('trying convert datatype of column to datetime using sql query')
    cursor.execute("SELECT  STR_TO_DATE(ActivityDate, '%m/%d/%Y') from fit_data.table1")
    for i in cursor:
        logging.info(i)
    logging.info('succescfully converted datattype of column using sql query') 
except Error as e:
    logging.error('problem while changing datatype to datetime using sql query',e)

In [20]:
#finding how many unique value present in this dataset

try:
    logging.info('finding unique id in dataset')
    df7 = df.Id.unique()
    logging.info('succesfully fined unique vale in dataset')
    logging.info(df7)
except Error as e:
    logging.error('facing problem while finding unique value in dataset')

In [17]:
try:
    logging.info('grouping data on bases of id column')
    df1 = df.groupby('Id').sum()
    df1.reset_index(inplace = True)
    logging.info('grouped data succesfully and store in variable name df1')
    df1.head(2)
except Error as e:
    logging.error('proble while grouping data')

In [25]:
# most active id in dataset
try:
    logging.info('finding most active id')
    var = df1[df1['Calories'] == max(df1['Calories'])]
    logging.info('succesfully find most active id')
    logging.info(var)
except Error as e:
    logging.error('problem while finding most active id in dataset',e)

In [22]:
#df1.head(2)

In [28]:
#checkin most active id using sql query
try:
    logging.info('trying to find most active id in dataset using sql query')
    cursor.execute("select id,max(Calories) from (select id,Calories  from fit_data.table1  group by id) as Calories")
    for i in cursor:
        logging.info(i)
    logging.info('succesfully find most active id using sql query')    
    
except Error as e:
    logging.error('problem while finding most active person in dataset using sql query',e)

In [29]:
#these person not logged thier activity
try:
    logging.info('finfing those person who not logged their activity')
    var1 = df1[df1['LoggedActivitiesDistance']== 0]['Id']
    logging.info('succusfuly finf those person who not logged their activity')
    logging.info(var1)
    
except Error as e:
    logging.error('problem while finding person who not logged their activity')

In [39]:
#finding laziest person id in dataset
try:
    logging.info('finding laziest person id')
    var3 = df1[df1['Calories']==min(df1['Calories'])][['Id','Calories']]
    logging.info('succesfully find laziest person in dataset')        
    logging.info(var3)
except Error as e:
    logging.error('problem while finding laziest person in dataset')


In [35]:
df1.head()

,Id,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories
0,1503960366,221170,144.540001,144.540001,0.0,52.89,12.07,79.39,0.00,681,301,4336,15387,34128
1,1624580081,80299,52.189999,52.189999,0.0,0.98,0.52,50.53,0.10,14,11,2300,24288,25705
2,1644430081,92748,67.490000,67.490000,0.0,11.34,20.30,35.80,0.08,148,435,2277,10342,29164
3,1844505072,43687,28.880000,28.880000,0.0,0.61,0.37,27.89,0.00,9,9,1902,12424,19391
4,1927972279,26170,18.130000,18.130000,0.0,0.00,0.74,17.39,0.00,0,20,1340,11435,27048


In [63]:
#checking healthy person according to burning of thier calories lets assume a person should burn 2000 calories per day
try:
    logging.info('finding healthy person according to burning of thier calories lets assume a person should burn 2000 calories per day')
    var5 = df[df['Calories']>=2000][['Id','Calories']]
    var6 = var5['Id'].unique()
    logging.info('find healthy person succesfully')
    logging.info(var6)
except Error as e:
    logging.error('problem while finding healthy person in dataset')
       
        

In [96]:
#finding third most active person in dataset with sql query:
try:
    logging.info('finding most active person with sql query')
    cursor.execute("select id , Calories from (select id , Calories from (select id,sum(Calories) as Calories from fit_data.table1 group by id) as Calories order by Calories DESC limit 3) as Calories order by Calories ASC LIMIT 1")
    for i in cursor:
        logging.info(i)
    logging.info('succesfully find most active person using sql query')    
except Error as e:
    logging.error('problem while finding most active person using sql query')

In [98]:
# total accumulate calories burn for a person 
try:
    logging.info('finding total accumulate calories burn for a person ')
    var8 = df1[['Id','Calories']]
    logging.info('find total calories of a person succesfully')
    logging.info(var8)
except Error as e:
    logging.error('problem while calculate total accumulate calories burn for a person ')

In [123]:
#finding the most active person in our dataset
try:
    logging.info('trying to find third most active person in dataset')
    var9 = df1[df1['Calories']==min(df1['Calories'].nlargest(3))]
    logging.info('find succesfully third most active person in dataset')
    logging.info(var9)
    
except Error as e:
    logging.error('problem while finding third most active person in dataset')

In [125]:
#finding fifth most laziest person in our dataset
try:
    logging.info('trying to find third most active person in dataset')
    var9 = df1[df1['Calories']==max(df1['Calories'].nsmallest(5))]
    logging.info('find succesfully third most active person in dataset')
    logging.info(var9)
    
except Error as e:
    logging.error('problem while finding third most active person in dataset')